# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [5]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-5e12566aa5-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Euro explorative search") 

Consider the following exploratory information need:

> investigate the Euro currency, which are the countries that currently use the Euro and which were their currencies before the Euro. Take a look also to other currencies and find which are the most used.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q38`| Italy   | node |





Also consider

```
wd:Q38 ?p ?obj .
```

is the BGP to retrieve all **properties of Italy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for currency

2. Return the countries which currently use Euro

3. Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

4. For each country that now uses Euro return which was the currencies used before

5. Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

6. Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

7. Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

## Task 1
Identify the BGP for currency

In [24]:
# query example
queryString = """
SELECT DISTINCT ?o ?oname ?p ?pname
WHERE {
   # bind something
   wd:Q38 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname.      
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q488981'), ('oname', 'European Bank for Reconstruction and Development'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
[('o', 'http://www.wikidata.org/entity/Q33973'), ('oname', 'Sicilian'), ('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pname', 'language used')]
[('o', 'http://www.wikidata.org/entity/Q7791'), ('oname', 'history of Italy'), ('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pname', 'history of topic')]
[('o', 'http://www.wikidata.org/entity/Q42876'), ('oname', 'flag of Italy'), ('p', 'http://www.wikidata.org/prop/direct/P163'), ('pname', 'flag')]
[('o', 'http://www.wikidata.org/entity/Q1273'), ('oname', 'Tuscany'), ('p', 'http://www.wikidata.org/prop/direct/P150'), ('pname', 'contains administrative territorial entity')]
[('o', 'http://www.wikidata.org/entity/Q33845'), ('oname', 'Neapolitan'), ('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pname', 'language used')]
[('o', 'htt

In [25]:
# searching for the same thing with filtering 
queryString = """
SELECT DISTINCT ?obj ?curname ?p ?name 
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?o sc:name ?curname .
   ?p sc:name ?name.
   
   
   FILTER(REGEX(?name ,"currency*"^^xsd:string))
      
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4916'), ('curname', 'software version identifier'), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
[('obj', 'http://www.wikidata.org/entity/Q4916'), ('curname', 'tabular software version'), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
[('obj', 'http://www.wikidata.org/entity/Q4916'), ('curname', 'has edition or translation'), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
[('obj', 'http://www.wikidata.org/entity/Q4916'), ('curname', 'Fonsi Nieto'), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
[('obj', 'http://www.wikidata.org/entity/Q4916'), ('curname', "Al Stewart's albums in chronological order"), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
[('obj', 'http://www.wikidata.org/entity/Q4916'), ('curname', 'Category:Adelina Lopes Vieira'), ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]

In [30]:
# ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?o ?oname ?s ?sname 
WHERE {
   # bind something
   ?s wdt:P38 ?o ;
      sc:name ?sname .
   ?o sc:name ?oname .
   
   FILTER(REGEX(?oname ,"euro*"^^xsd:string))   
      
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro'), ('s', 'http://www.wikidata.org/entity/Q1628531'), ('sname', 'Quintanar de la Orden')]
[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro'), ('s', 'http://www.wikidata.org/entity/Q673727'), ('sname', 'Neudau')]
[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro'), ('s', 'http://www.wikidata.org/entity/Q28124876'), ('sname', 'Ilmspan')]
[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro'), ('s', 'http://www.wikidata.org/entity/Q11959'), ('sname', 'Alicante')]
[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro'), ('s', 'http://www.wikidata.org/entity/Q25362'), ('sname', 'Saint Barthélemy')]
[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro'), ('s', 'http://www.wikidata.org/entity/Q17054'), ('sname', 'Martinique')]
[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro'), ('s', 'http://www.wikidata.org/entity/Q656699'), ('sname', 'Montsoreau'

Final query for this task

In [31]:
# write your final query
# ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?o ?oname ?s ?sname 
WHERE {
   # bind something
   ?s wdt:P38 ?o ;
      sc:name ?sname .
   ?o sc:name ?oname .
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8428168'), ('oname', 'stored-value card'), ('s', 'http://www.wikidata.org/entity/Q7296276'), ('sname', 'Rav-Kav')]
[('o', 'http://www.wikidata.org/entity/Q4591'), ('oname', 'Cape Verdean escudo'), ('s', 'http://www.wikidata.org/entity/Q1011'), ('sname', 'Cape Verde')]
[('o', 'http://www.wikidata.org/entity/Q4591'), ('oname', 'Cape Verdean escudo'), ('s', 'http://www.wikidata.org/entity/Q4393598'), ('sname', 'Republic of the Cape Verde Islands')]
[('o', 'http://www.wikidata.org/entity/Q22907080'), ('oname', 'Carthaginian currency'), ('s', 'http://www.wikidata.org/entity/Q2429397'), ('sname', 'Ancient Carthage')]
[('o', 'http://www.wikidata.org/entity/Q1246307'), ('oname', 'Tornesel'), ('s', 'http://www.wikidata.org/entity/Q30890672'), ('sname', 'Kingdom of Sicily')]
[('o', 'http://www.wikidata.org/entity/Q1542162'), ('oname', 'Tarì'), ('s', 'http://www.wikidata.org/entity/Q30890672'), ('sname', 'Kingdom of Sicily')]
[('o', 'http://www.wiki

## Task 2
Return the countries which currently use Euro

In [32]:
# here it's returned the cities and towns which use euro
# ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?s ?sname ?oname  
WHERE {
   # bind something
   ?s wdt:P38 ?o ;
      sc:name ?sname .
   ?o sc:name ?oname .
   
   FILTER(REGEX(?oname ,"euro*"^^xsd:string))   
      
}


LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1628531'), ('sname', 'Quintanar de la Orden'), ('oname', 'euro')]
[('s', 'http://www.wikidata.org/entity/Q673727'), ('sname', 'Neudau'), ('oname', 'euro')]
[('s', 'http://www.wikidata.org/entity/Q28124876'), ('sname', 'Ilmspan'), ('oname', 'euro')]
[('s', 'http://www.wikidata.org/entity/Q11959'), ('sname', 'Alicante'), ('oname', 'euro')]
[('s', 'http://www.wikidata.org/entity/Q25362'), ('sname', 'Saint Barthélemy'), ('oname', 'euro')]
[('s', 'http://www.wikidata.org/entity/Q17054'), ('sname', 'Martinique'), ('oname', 'euro')]
[('s', 'http://www.wikidata.org/entity/Q656699'), ('sname', 'Montsoreau'), ('oname', 'euro')]
[('s', 'http://www.wikidata.org/entity/Q624511'), ('sname', 'Aracena'), ('oname', 'euro')]
[('s', 'http://www.wikidata.org/entity/Q1246'), ('sname', 'Kosovo'), ('oname', 'euro')]
[('s', 'http://www.wikidata.org/entity/Q126125'), ('sname', 'Saint Martin (French part)'), ('oname', 'euro')]
[('s', 'http://www.wikidata.org/entit

In [8]:
#not relevant 
# [('s', 'http://www.wikidata.org/entity/Q183'), ('sname', 'Germany'), ('oname', 'euro')]
# [('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname  
WHERE {
   # bind something
   wd:Q183 ?p ?o .
   ?p sc:name ?pname .
   ?o sc:name ?oname .    
}


LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('pname', 'diplomatic relation'), ('o', 'http://www.wikidata.org/entity/Q1005'), ('oname', 'The Gambia')]
[('p', 'http://www.wikidata.org/prop/direct/P208'), ('pname', 'executive body'), ('o', 'http://www.wikidata.org/entity/Q1005903'), ('oname', 'federal level of Germany')]
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('pname', 'diplomatic relation'), ('o', 'http://www.wikidata.org/entity/Q1008'), ('oname', 'Ivory Coast')]
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('pname', 'diplomatic relation'), ('o', 'http://www.wikidata.org/entity/Q1011'), ('oname', 'Cape Verde')]
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('pname', 'diplomatic relation'), ('o', 'http://www.wikidata.org/entity/Q1013'), ('oname', 'Lesotho')]
[('p', 'http://www.wikidata.org/prop/direct/P530'), ('pname', 'diplomatic relation'), ('o', 'http://www.wikidata.org/entity/Q1014'), ('oname', 'Liberia')]
[('p', 'http://www.wikidata.org/prop/

In [24]:
# [('s', 'http://www.wikidata.org/entity/Q183'), ('sname', 'Germany'), ('oname', 'euro')]
#[('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {
   # bind something
   ?s ?p wd:Q183 .
   ?p sc:name ?pname .
   ?s sc:name ?sname .    

   FILTER(REGEX(?pname ,"country*"^^xsd:string)) 

}


LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pname', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P205'), ('pname', 'basin country')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin')]
[('p', 'http://www.wikidata.org/prop/direct/P8047'), ('pname', 'country of registry')]
6


In [14]:
#not relevant
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
queryString = """
SELECT DISTINCT ?s ?sname ?o ?oname
WHERE {
   # bind something
   ?s wdt:P17 ?o .
   ?o sc:name ?oname .
   ?s sc:name ?sname .    



}


LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2535173'), ('sname', 'South Vietnam Air Force'), ('o', 'http://www.wikidata.org/entity/Q180573'), ('oname', 'South Vietnam')]
[('s', 'http://www.wikidata.org/entity/Q16171015'), ('sname', 'Arshamuni'), ('o', 'http://www.wikidata.org/entity/Q2981019'), ('oname', 'Ostikanate of Arminiya')]
[('s', 'http://www.wikidata.org/entity/Q10889269'), ('sname', 'Aolai'), ('o', 'http://www.wikidata.org/entity/Q10889269'), ('oname', 'Aolai')]
[('s', 'http://www.wikidata.org/entity/Q17029014'), ('sname', 'Mount Huaguo'), ('o', 'http://www.wikidata.org/entity/Q10889269'), ('oname', 'Aolai')]
[('s', 'http://www.wikidata.org/entity/Q6207845'), ('sname', 'Jodhpur State'), ('o', 'http://www.wikidata.org/entity/Q6124144'), ('oname', 'Jaipur State')]
[('s', 'http://www.wikidata.org/entity/Q21000095'), ('sname', 'Plaisance'), ('o', 'http://www.wikidata.org/entity/Q7646305'), ('oname', 'Surinam')]
[('s', 'http://www.wikidata.org/entity/Q1247214'), ('sname', 'Soba

In [22]:
#again returned cities
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')
#[ ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   # bind something
   ?s wdt:P38 wd:Q4916 ;
      sc:name ?sname .
FILTER EXISTS {
  # bind something
   ?s wdt:P17 ?o .
}      
}


LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1628531'), ('sname', 'Quintanar de la Orden')]
[('s', 'http://www.wikidata.org/entity/Q673727'), ('sname', 'Neudau')]
[('s', 'http://www.wikidata.org/entity/Q28124876'), ('sname', 'Ilmspan')]
[('s', 'http://www.wikidata.org/entity/Q11959'), ('sname', 'Alicante')]
[('s', 'http://www.wikidata.org/entity/Q25362'), ('sname', 'Saint Barthélemy')]
[('s', 'http://www.wikidata.org/entity/Q17054'), ('sname', 'Martinique')]
[('s', 'http://www.wikidata.org/entity/Q656699'), ('sname', 'Montsoreau')]
[('s', 'http://www.wikidata.org/entity/Q624511'), ('sname', 'Aracena')]
[('s', 'http://www.wikidata.org/entity/Q1246'), ('sname', 'Kosovo')]
[('s', 'http://www.wikidata.org/entity/Q126125'), ('sname', 'Saint Martin (French part)')]
10


In [27]:
# Changed the ?s to ?o 
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')
#[ ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   # bind something
   ?o wdt:P38 wd:Q4916 ;
      sc:name ?sname .
FILTER EXISTS {
  # bind something
   ?s wdt:P17 ?o .
}      
}


LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'Alicante')]
[('sname', 'Saint Barthélemy')]
[('sname', 'Martinique')]
[('sname', 'Kosovo')]
[('sname', 'Saint Martin (French part)')]
[('sname', 'French Southern and Antarctic Lands')]
[('sname', 'France')]
[('sname', 'Germany')]
[('sname', 'Estonia')]
[('sname', 'Latvia')]
[('sname', 'Slovakia')]
[('sname', 'Slovenia')]
[('sname', 'Andorra')]
[('sname', 'Cyprus')]
[('sname', 'Malta')]
[('sname', 'Monaco')]
[('sname', 'Montenegro')]
[('sname', 'Vatican City')]
[('sname', 'San Marino')]
[('sname', 'Ireland')]
[('sname', 'Spain')]
[('sname', 'Kingdom of the Netherlands')]
[('sname', 'Belgium')]
[('sname', 'Luxembourg')]
[('sname', 'Finland')]
[('sname', 'Lithuania')]
[('sname', 'French Guiana')]
[('sname', 'Italy')]
[('sname', 'Austria')]
[('sname', 'Greece')]
[('sname', 'Portugal')]
[('sname', 'European Union')]
[('sname', 'Netherlands')]
[('sname', 'Guadeloupe')]
[('sname', 'Réunion')]
[('sname', 'Åland')]
[('sname', 'Serbia and Montenegro')]
[('sname', 'Saint Pierr

In [28]:
# get the info from the first query task_1
# [('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')
#[ ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   # bind something
   ?s wdt:P38 wd:Q4916 ;
      sc:name ?sname .
FILTER EXISTS {
  # bind something
   ?s wdt:P463 wd:Q458 .
}      
}


LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('sname', 'France')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('sname', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('sname', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('sname', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('sname', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('sname', 'Slovenia')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('sname', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('sname', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('sname', 'Ireland')]
[('s', 'http://www.wikidata.org/entity/Q29'), ('sname', 'Spain')]
[('s', 'http://www.wikidata.org/entity/Q29999'), ('sname', 'Kingdom of the Netherlands')]
[('s', 'http://www.wikidata.org/entity/Q31'), ('sname', 'Belgium')]
[('s', 'http://www.wikidata.org/entity/Q32'), ('sname', 'Luxembourg')]
[('s', 'http://www.wikidata.org/entity/Q33'), ('sname', 'Finland')]
[('s', 'http

Final query for this task

In [29]:
# write your final query
#countries which currently use euro
# [('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')
#[ ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   # country which currency euro
   ?s wdt:P38 wd:Q4916 ;
      sc:name ?sname .
      
FILTER EXISTS {
  # country which is the member of EU
   ?s wdt:P463 wd:Q458 .
    }  

}


LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('sname', 'France')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('sname', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('sname', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('sname', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('sname', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('sname', 'Slovenia')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('sname', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('sname', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('sname', 'Ireland')]
[('s', 'http://www.wikidata.org/entity/Q29'), ('sname', 'Spain')]
[('s', 'http://www.wikidata.org/entity/Q29999'), ('sname', 'Kingdom of the Netherlands')]
[('s', 'http://www.wikidata.org/entity/Q31'), ('sname', 'Belgium')]
[('s', 'http://www.wikidata.org/entity/Q32'), ('sname', 'Luxembourg')]
[('s', 'http://www.wikidata.org/entity/Q33'), ('sname', 'Finland')]
[('s', 'http

## Task 3
Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

In [47]:
# get the info from the first query task_1
# [('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]

queryString = """
SELECT DISTINCT ?s ?sname ?o ?oname  
WHERE {
   # bind something
   ?s wdt:P463 ?o ;
      sc:name ?sname .
   ?o sc:name ?oname .
   
   FILTER(REGEX(?oname ,"European Union*"^^xsd:string))   
      
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q183'), ('sname', 'Germany'), ('o', 'http://www.wikidata.org/entity/Q189946'), ('oname', 'Western European Union')]
[('s', 'http://www.wikidata.org/entity/Q38'), ('sname', 'Italy'), ('o', 'http://www.wikidata.org/entity/Q189946'), ('oname', 'Western European Union')]
[('s', 'http://www.wikidata.org/entity/Q142'), ('sname', 'France'), ('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union')]
[('s', 'http://www.wikidata.org/entity/Q145'), ('sname', 'United Kingdom'), ('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('sname', 'Germany'), ('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('sname', 'Estonia'), ('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('sname', 'Latvia'), ('o', 'http://www.wikidata.org/enti

In [51]:
# ('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union')]
# [('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
queryString = """
SELECT DISTINCT ?s ?sname  
WHERE {
   # bind something
   ?s wdt:P463 wd:Q458 ;
      sc:name ?sname .
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q142'), ('sname', 'France')]
[('s', 'http://www.wikidata.org/entity/Q145'), ('sname', 'United Kingdom')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('sname', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('sname', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q211'), ('sname', 'Latvia')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('sname', 'Czech Republic')]
[('s', 'http://www.wikidata.org/entity/Q214'), ('sname', 'Slovakia')]
[('s', 'http://www.wikidata.org/entity/Q215'), ('sname', 'Slovenia')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('sname', 'Romania')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('sname', 'Bulgaria')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('sname', 'Croatia')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('sname', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q233'), ('sname', 'Malta')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('sname', 'Ireland')]
[('s', 'http://

In [52]:
# ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?o ?oname  
WHERE {
   # bind something
   ?s wdt:P38 ?o ;
      sc:name ?sname .
   ?o sc:name ?oname .
   
   FILTER(REGEX(?oname ,"euro*"^^xsd:string))   
      
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')]
1


In [3]:
# Countries that are in Europian Union and not using euro currency 
#[('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')]
queryString = """
SELECT DISTINCT ?s ?sname  
WHERE {
    {
   # bind something
   ?s wdt:P463 wd:Q458 ;
      sc:name ?sname   
      
}

MINUS{
  # bind something
   ?s wdt:P38 wd:Q4916  
}
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q145'), ('sname', 'United Kingdom')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('sname', 'Czech Republic')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('sname', 'Romania')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('sname', 'Bulgaria')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('sname', 'Croatia')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('sname', 'Hungary')]
[('s', 'http://www.wikidata.org/entity/Q34'), ('sname', 'Sweden')]
[('s', 'http://www.wikidata.org/entity/Q35'), ('sname', 'Denmark')]
[('s', 'http://www.wikidata.org/entity/Q36'), ('sname', 'Poland')]
9


In [37]:
# [('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
#[('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')]
queryString = """
SELECT DISTINCT ?s ?sname ?oname
WHERE {
    
   # bind something
   ?s wdt:P463 wd:Q458 ;
      sc:name ?sname .  
      
FILTER NOT EXISTS {
    ?s wdt:P38 wd:Q4916 
}
OPTIONAL{
    ?s wdt:P38 ?o .
    ?o sc:name ?oname
    
}



}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q36'), ('sname', 'Poland'), ('oname', 'złoty')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('sname', 'Bulgaria'), ('oname', 'Bulgarian lev')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('sname', 'Hungary'), ('oname', 'forint')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('sname', 'Romania'), ('oname', 'Romanian leu')]
[('s', 'http://www.wikidata.org/entity/Q145'), ('sname', 'United Kingdom'), ('oname', 'pound sterling')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('sname', 'Croatia'), ('oname', 'Croatian kuna')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('sname', 'Czech Republic'), ('oname', 'Czech koruna')]
[('s', 'http://www.wikidata.org/entity/Q35'), ('sname', 'Denmark'), ('oname', 'Danish krone')]
[('s', 'http://www.wikidata.org/entity/Q34'), ('sname', 'Sweden'), ('oname', 'Swedish krona')]
9


Final query for this task

In [38]:
# write your final query
#countries belonging to the European Union, with a currency different from the Euro 
queryString = """
SELECT DISTINCT ?s ?sname ?oname ?o
WHERE {
   # country member of EU
   ?s wdt:P463 wd:Q458 ;
      sc:name ?sname .  
      
FILTER NOT EXISTS {
    #country currency euro
    ?s wdt:P38 wd:Q4916 
    }

OPTIONAL{
    #looking which country uses which currency
    ?s wdt:P38 ?o .
    ?o sc:name ?oname
    }
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q36'), ('sname', 'Poland'), ('oname', 'złoty'), ('o', 'http://www.wikidata.org/entity/Q123213')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('sname', 'Bulgaria'), ('oname', 'Bulgarian lev'), ('o', 'http://www.wikidata.org/entity/Q172540')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('sname', 'Hungary'), ('oname', 'forint'), ('o', 'http://www.wikidata.org/entity/Q47190')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('sname', 'Romania'), ('oname', 'Romanian leu'), ('o', 'http://www.wikidata.org/entity/Q131645')]
[('s', 'http://www.wikidata.org/entity/Q145'), ('sname', 'United Kingdom'), ('oname', 'pound sterling'), ('o', 'http://www.wikidata.org/entity/Q25224')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('sname', 'Croatia'), ('oname', 'Croatian kuna'), ('o', 'http://www.wikidata.org/entity/Q26360')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('sname', 'Czech Republic'), ('oname', 'Czech koruna'), ('o', 'http://www.wikidata

## Task 4
For each country that now uses Euro return which was the currencies used before

In [24]:
# [('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')
#[ ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?o ?oname ?pname ?oname
WHERE {
   # country which currency euro
   wd:Q29 ?p ?o .
      ?p sc:name ?pname .
      ?o sc:name ?oname .
      
      FILTER(REGEX(?pname ,"currencies used before*"^^xsd:string))   

}


LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 42S22 Error SQ149: The column \'oname\' was specified multiple times for \'s_25_6_rbc\'\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-5e12566aa5-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT DISTINCT ?o ?oname ?pname ?oname\nWHERE {\n   # country which currency euro\n   wd:Q29 ?p ?o .\n      ?p sc:name ?pname .\n      ?o sc:name ?oname .\n      \n      FILTER(REGEX(?pname ,"currencies used before*"^^xsd:string))   \n\n}\n\n\nLIMIT 20\n'


In [25]:
# write your queries
# [('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')
#[ ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
#from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
#| `wdt:P31`     | instance of 
# wd:Q8142 -> Currency
queryString = """
SELECT DISTINCT ?l ?noteuro ?pname 
#?pname  ?currencyname ?ins ?incname ?noteuro
WHERE {
  # country that used different currencies from now
     wd:Q4916 wdt:P17 ?p .
     #?p wdt:P38 ?cur.
     ?l wdt:P31 wd:Q8142;
         wdt:P17 ?p.
     
     
    #?cur sc:name ?currencyname .
    #?ins sc:name ?incname.
    ?l sc:name ?pname.
    ?p sc:name ?noteuro.
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('l', 'http://www.wikidata.org/entity/Q3658911'), ('noteuro', 'Malta'), ('pname', 'Carlino')]
[('l', 'http://www.wikidata.org/entity/Q3658911'), ('noteuro', 'Italy'), ('pname', 'Carlino')]
[('l', 'http://www.wikidata.org/entity/Q16068'), ('noteuro', 'Germany'), ('pname', 'Deutsche Mark')]
[('l', 'http://www.wikidata.org/entity/Q203354'), ('noteuro', 'Finland'), ('pname', 'Finnish markka')]
[('l', 'http://www.wikidata.org/entity/Q214393'), ('noteuro', 'France'), ('pname', 'CFP Franc')]
[('l', 'http://www.wikidata.org/entity/Q2356117'), ('noteuro', 'France'), ('pname', 'New Caledonian franc')]
[('l', 'http://www.wikidata.org/entity/Q2356178'), ('noteuro', 'France'), ('pname', 'French Polynesian franc')]
[('l', 'http://www.wikidata.org/entity/Q4916'), ('noteuro', 'France'), ('pname', 'euro')]
[('l', 'http://www.wikidata.org/entity/Q4916'), ('noteuro', 'Germany'), ('pname', 'euro')]
[('l', 'http://www.wikidata.org/entity/Q4916'), ('noteuro', 'Estonia'), ('pname', 'euro')]
[('l', '

Final query for this task

In [33]:
# write your final query
# write your queries
# [('o', 'http://www.wikidata.org/entity/Q458'), ('oname', 'European Union'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')
#[ ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
#from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
#| `wdt:P31`     | instance of 
# wd:Q8142 -> Currency
queryString = """
SELECT DISTINCT ?IRI ?country ?prev_currency
#?pname  ?currencyname ?ins ?prev_currency ?country
WHERE {
  # country that used different currencies from now
     wd:Q4916 wdt:P17 ?p .
     #?p wdt:P38 ?cur.
     ?IRI wdt:P31 wd:Q8142;
         wdt:P17 ?p.
     
     
    #?cur sc:name ?currencyname .
    #?ins sc:name ?incname.
    ?IRI sc:name ?prev_currency.
    ?p sc:name ?country.
    
    Filter(str(?prev_currency) != "euro")
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q2356178'), ('country', 'France'), ('prev_currency', 'French Polynesian franc')]
[('IRI', 'http://www.wikidata.org/entity/Q214393'), ('country', 'France'), ('prev_currency', 'CFP Franc')]
[('IRI', 'http://www.wikidata.org/entity/Q2616592'), ('country', 'France'), ('prev_currency', 'French West African franc')]
[('IRI', 'http://www.wikidata.org/entity/Q2356117'), ('country', 'France'), ('prev_currency', 'New Caledonian franc')]
[('IRI', 'http://www.wikidata.org/entity/Q184172'), ('country', 'France'), ('prev_currency', 'French franc')]
[('IRI', 'http://www.wikidata.org/entity/Q16068'), ('country', 'Germany'), ('prev_currency', 'Deutsche Mark')]
[('IRI', 'http://www.wikidata.org/entity/Q129983'), ('country', 'Estonia'), ('prev_currency', 'Estonian kroon')]
[('IRI', 'http://www.wikidata.org/entity/Q133011'), ('country', 'Latvia'), ('prev_currency', 'Latvian lats')]
[('IRI', 'http://www.wikidata.org/entity/Q178874'), ('country', 'Slovakia'),

## Task 5
Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

In [100]:
# write your queries
#[ ('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
queryString = """
SELECT DISTINCT ?s ?sname ?o ?oname
WHERE {
   #
   ?s wdt:P38 ?o ;
      sc:name ?sname .
      ?o sc:name ?oname .
      
    FILTER(REGEX(?oname ,"Dollar*"^^xsd:string)) 
    

}


LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q170588'), ('sname', 'Republic of Texas'), ('o', 'http://www.wikidata.org/entity/Q3712659'), ('oname', 'Texas Dollar')]
[('s', 'http://www.wikidata.org/entity/Q1123960'), ('sname', 'Conch Republic'), ('o', 'http://www.wikidata.org/entity/Q19831166'), ('oname', 'Conch Dollar')]
[('s', 'http://www.wikidata.org/entity/Q106827'), ('sname', 'Second Life'), ('o', 'http://www.wikidata.org/entity/Q100966169'), ('oname', 'Linden Dollar')]
3


In [103]:
#from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
queryString = """
SELECT DISTINCT ?o ?oname 
WHERE {
   #
   ?s wdt:P38 ?o ;
      sc:name ?sname .
      ?o sc:name ?oname .

    FILTER(REGEX(?oname ,"dollar*"^^xsd:string)) 
    
OPTIONAL {
    #only country
    ?s wdt:P17 ?o ;
        sc:name ?sname .
    }
}


LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4917'), ('oname', 'United States dollar')]
[('o', 'http://www.wikidata.org/entity/Q2328868'), ('oname', 'Hawaiian dollar')]
[('o', 'http://www.wikidata.org/entity/Q686163'), ('oname', 'Confederate States dollar')]
[('o', 'http://www.wikidata.org/entity/Q261447'), ('oname', 'Straits dollar')]
[('o', 'http://www.wikidata.org/entity/Q15039722'), ('oname', 'spanish dollar')]
[('o', 'http://www.wikidata.org/entity/Q259502'), ('oname', 'Australian dollar')]
[('o', 'http://www.wikidata.org/entity/Q1607372'), ('oname', 'Kiautschou dollar')]
[('o', 'http://www.wikidata.org/entity/Q2344772'), ('oname', 'Malayan dollar')]
[('o', 'http://www.wikidata.org/entity/Q392858'), ('oname', 'Rhodesian dollar')]
[('o', 'http://www.wikidata.org/entity/Q209792'), ('oname', 'Jamaican dollar')]
10


Final query for this task

In [3]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?oname 
WHERE {
   #currency
   ?s wdt:P38 ?o ;
      sc:name ?sname .
      ?o sc:name ?oname .

    FILTER(REGEX(?oname ,"[Dd]ollar*"^^xsd:string)) 
    
OPTIONAL {
    #only country
    ?s wdt:P17 ?o ;
        sc:name ?sname .
    }
}


LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4917'), ('oname', 'United States dollar')]
[('o', 'http://www.wikidata.org/entity/Q2328868'), ('oname', 'Hawaiian dollar')]
[('o', 'http://www.wikidata.org/entity/Q686163'), ('oname', 'Confederate States dollar')]
[('o', 'http://www.wikidata.org/entity/Q261447'), ('oname', 'Straits dollar')]
[('o', 'http://www.wikidata.org/entity/Q15039722'), ('oname', 'spanish dollar')]
[('o', 'http://www.wikidata.org/entity/Q259502'), ('oname', 'Australian dollar')]
[('o', 'http://www.wikidata.org/entity/Q1607372'), ('oname', 'Kiautschou dollar')]
[('o', 'http://www.wikidata.org/entity/Q3712659'), ('oname', 'Texas Dollar')]
[('o', 'http://www.wikidata.org/entity/Q2344772'), ('oname', 'Malayan dollar')]
[('o', 'http://www.wikidata.org/entity/Q392858'), ('oname', 'Rhodesian dollar')]
[('o', 'http://www.wikidata.org/entity/Q209792'), ('oname', 'Jamaican dollar')]
[('o', 'http://www.wikidata.org/entity/Q650834'), ('oname', 'Japanese government-issued dollar

## Task 6
Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

In [3]:
# write your queries
##from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
queryString = """
SELECT DISTINCT ?sname ?oname 
WHERE {
   #looking for properties of country predicate
   ?s wdt:P17 ?o ;
      sc:name ?sname .
      ?o sc:name ?oname .
}


LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'South Vietnam Air Force'), ('oname', 'South Vietnam')]
[('sname', 'religion in Greenland'), ('oname', 'Greenland')]
[('sname', 'Freedom and Reform Party'), ('oname', 'Turkish Republic of Northern Cyprus')]
[('sname', 'Military of Northern Cyprus'), ('oname', 'Turkish Republic of Northern Cyprus')]
[('sname', 'flora of the Dominican Republic'), ('oname', 'Dominican Republic')]
[('sname', 'Dominican Republic at the Pan American Games'), ('oname', 'Dominican Republic')]
[('sname', 'ambassador of France to Guinea-Bissau'), ('oname', 'Guinea-Bissau')]
[('sname', 'Alliance for Democracy and Development'), ('oname', 'Cameroon')]
[('sname', '1997 in Cape Verde'), ('oname', 'Cape Verde')]
[('sname', '2000 in Cape Verde'), ('oname', 'Cape Verde')]
10


In [11]:
# write your queries
# binding the country and currency properties
#from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
queryString = """
SELECT DISTINCT ?countryname ?currencyname 
WHERE {

   #looking for properties of country predicate
   ?s wdt:P17 ?country .
   ?country wdt:P38 ?currency ;
      sc:name ?countryname .
      ?currency sc:name ?currencyname .

}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('countryname', 'First Republic of South Korea'), ('currencyname', 'South Korean won')]
[('countryname', 'Nyasaland'), ('currencyname', 'Southern Rhodesian pound')]
[('countryname', 'Fourth Republic of South Korea'), ('currencyname', 'South Korean won')]
[('countryname', 'Bophuthatswana'), ('currencyname', 'South African rand')]
[('countryname', 'East Francia'), ('currencyname', 'solidus')]
[('countryname', 'East Francia'), ('currencyname', 'Tremissis')]
[('countryname', 'East Francia'), ('currencyname', 'denarius')]
[('countryname', 'East Francia'), ('currencyname', 'pfennig')]
[('countryname', 'Kingdom of Hawaiʻi'), ('currencyname', 'Hawaiian dollar')]
[('countryname', 'Kingdom of Hawaiʻi'), ('currencyname', 'United States dollar')]
[('countryname', 'South Vietnam'), ('currencyname', 'South Vietnamese đồng')]
[('countryname', "People's Republic of Korea"), ('currencyname', 'South Korean won')]
[('countryname', 'Uzbek Soviet Socialist Republic'), ('currencyname', 'Soviet rubl

In [17]:
# write your queries
# counting the contries using the same currency
#from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
queryString = """
SELECT DISTINCT  ?currencyname (COUNT(?countryname) AS ?countryCount) #counting in how many countries the currency used
WHERE {

   #looking for properties of country predicate
   ?s wdt:P17 ?country .
   ?country wdt:P38 ?currency ;
      sc:name ?countryname .
      ?currency sc:name ?currencyname .

}

GROUP BY ?currencyname

ORDER BY DESC (?countryCount)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('currencyname', 'euro'), ('countryCount', '3645924')]
[('currencyname', 'United States dollar'), ('countryCount', '1483878')]
[('currencyname', 'pound sterling'), ('countryCount', '1016077')]
[('currencyname', 'renminbi'), ('countryCount', '895554')]
[('currencyname', 'Indian rupee'), ('countryCount', '764912')]
[('currencyname', 'Russian ruble'), ('countryCount', '409701')]
[('currencyname', 'Norwegian krone'), ('countryCount', '400320')]
[('currencyname', 'Swedish krona'), ('countryCount', '389610')]
[('currencyname', 'Canadian dollar'), ('countryCount', '379924')]
[('currencyname', 'Mexican peso'), ('countryCount', '360834')]
10


In [21]:
#want to check is the query really represents only countries are using euro or something else too 
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')
queryString = """
SELECT DISTINCT  ?countryname ?currencyname #(COUNT(?countryname) AS ?countryCount) #counting in how many countries the currency used
WHERE {

   #looking for properties of country predicate
   ?s wdt:P17 ?country .
   ?country wdt:P38 wd:Q4916 ;
      sc:name ?countryname .
      wd:Q4916 sc:name ?currencyname .

}

GROUP BY ?currencyname

#ORDER BY DESC (?countryCount)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('countryname', 'Verdis'), ('currencyname', 'euro')]
[('countryname', 'Montenegro'), ('currencyname', 'euro')]
[('countryname', 'Latvia'), ('currencyname', 'euro')]
[('countryname', 'Monaco'), ('currencyname', 'euro')]
[('countryname', 'Catalan Republic'), ('currencyname', 'euro')]
[('countryname', 'France'), ('currencyname', 'euro')]
[('countryname', 'Belgium'), ('currencyname', 'euro')]
[('countryname', 'Greece'), ('currencyname', 'euro')]
[('countryname', 'French Southern and Antarctic Lands'), ('currencyname', 'euro')]
[('countryname', 'Slovakia'), ('currencyname', 'euro')]
[('countryname', 'Malta'), ('currencyname', 'euro')]
[('countryname', 'Austria'), ('currencyname', 'euro')]
[('countryname', 'Martinique'), ('currencyname', 'euro')]
[('countryname', 'Kosovo'), ('currencyname', 'euro')]
[('countryname', 'Estonia'), ('currencyname', 'euro')]
[('countryname', 'Andorra'), ('currencyname', 'euro')]
[('countryname', 'Kingdom of the Netherlands'), ('currencyname', 'euro')]
[(

In [27]:
#double checking the predicate country, maybe there any others
queryString = """
SELECT DISTINCT ?p ?pname ?oname 
WHERE {
   #looking for properties of country predicate
   wd:Q38 ?p ?o .
     ?p sc:name ?pname .
     ?o sc:name ?oname .
     
FILTER(REGEX(?pname , "[Cc]ountry*"^^xsd:string)) .

}


LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('oname', 'Italy')]
1


In [32]:
#checking how many countries using euro
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')
queryString = """
SELECT DISTINCT ?countryname ?currencyname #(COUNT(?country) AS ?countryCount) #counting in how many countries the currency used
WHERE {

   #looking for properties of country predicate
   ?city wdt:P17 ?country .
   #looking for properties of currency predicate
   ?country wdt:P38 ?currency ;
      sc:name ?countryname .
      ?currency sc:name ?currencyname .
      
FILTER(REGEX(?currencyname , "[Ee]uro*"^^xsd:string)) .

}

GROUP BY ?currencyname

#ORDER BY DESC (?countryCount)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('countryname', 'Verdis'), ('currencyname', 'euro')]
[('countryname', 'Montenegro'), ('currencyname', 'euro')]
[('countryname', 'Monaco'), ('currencyname', 'euro')]
[('countryname', 'Latvia'), ('currencyname', 'euro')]
[('countryname', 'Catalan Republic'), ('currencyname', 'euro')]
[('countryname', 'Belgium'), ('currencyname', 'euro')]
[('countryname', 'Greece'), ('currencyname', 'euro')]
[('countryname', 'France'), ('currencyname', 'euro')]
[('countryname', 'Austria'), ('currencyname', 'euro')]
[('countryname', 'French Southern and Antarctic Lands'), ('currencyname', 'euro')]
[('countryname', 'Slovakia'), ('currencyname', 'euro')]
[('countryname', 'Malta'), ('currencyname', 'euro')]
[('countryname', 'European Economic Community'), ('currencyname', 'European Currency Unit')]
[('countryname', 'Martinique'), ('currencyname', 'euro')]
[('countryname', 'Kosovo'), ('currencyname', 'euro')]
[('countryname', 'Guadeloupe'), ('currencyname', 'euro')]
[('countryname', 'Åland'), ('curren

In [7]:
#added distinct to the countries for the quiry not to count repeated names of the countries
#[('o', 'http://www.wikidata.org/entity/Q4916'), ('oname', 'euro')
queryString = """
SELECT DISTINCT ?currencyname (COUNT(distinct ?country) AS ?countryCount) #counting in how many countries the currency used
WHERE {

   #looking for properties of country predicate
   ?city wdt:P17 ?country .
   #looking for properties of currency predicate
   ?country wdt:P38 ?currency ;
      sc:name ?countryname .
      ?currency sc:name ?currencyname .
      
FILTER(REGEX(?currencyname , "[Ee]uro*"^^xsd:string)) .

}

GROUP BY ?currencyname

ORDER BY DESC (?countryCount)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currencyname', 'euro'), ('countryCount', '40')]
[('currencyname', 'European Currency Unit'), ('countryCount', '1')]
2


In [8]:
# counting the contries using the same currency
#from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
queryString = """
SELECT DISTINCT  ?currencyname (COUNT(distinct ?countryname) AS ?countryCount) #counting in how many countries the currency used
WHERE {

   #looking for properties of country predicate
   ?s wdt:P17 ?country .
   ?country wdt:P38 ?currency ;
      sc:name ?countryname .
      ?currency sc:name ?currencyname .

}

GROUP BY ?currencyname

ORDER BY DESC (?countryCount)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('currencyname', 'euro'), ('countryCount', '40')]
[('currencyname', 'pound sterling'), ('countryCount', '28')]
[('currencyname', 'United States dollar'), ('countryCount', '22')]
[('currencyname', 'Soviet ruble'), ('countryCount', '20')]
[('currencyname', 'French franc'), ('countryCount', '18')]
[('currencyname', 'dinar'), ('countryCount', '13')]
[('currencyname', 'peseta'), ('countryCount', '11')]
[('currencyname', 'Egyptian pound'), ('countryCount', '11')]
[('currencyname', 'Canadian dollar'), ('countryCount', '10')]
[('currencyname', 'ancient Chinese coinage'), ('countryCount', '10')]
10


In [75]:
# counting the contries using the same currency and filering those which currency used less than 2 times 
#from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
queryString = """
SELECT DISTINCT ?currencyname (COUNT(distinct ?countryname) AS ?countryCount) #counting in how many countries the currency used
WHERE {

   #looking for properties of country predicate
   ?s wdt:P17 ?country .
   ?country wdt:P38 ?currency .
   ?currency sc:name ?currencyname.
    ?country sc:name ?countryname .   

}

GROUP BY ?currencyname
HAVING(COUNT(distinct ?countryname)>=2)
ORDER BY DESC (?countryCount)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('currencyname', 'euro'), ('countryCount', '40')]
[('currencyname', 'pound sterling'), ('countryCount', '28')]
[('currencyname', 'United States dollar'), ('countryCount', '22')]
[('currencyname', 'Soviet ruble'), ('countryCount', '20')]
[('currencyname', 'French franc'), ('countryCount', '18')]
[('currencyname', 'dinar'), ('countryCount', '13')]
[('currencyname', 'peseta'), ('countryCount', '11')]
[('currencyname', 'Egyptian pound'), ('countryCount', '11')]
[('currencyname', 'Canadian dollar'), ('countryCount', '10')]
[('currencyname', 'ancient Chinese coinage'), ('countryCount', '10')]
[('currencyname', 'Russian ruble'), ('countryCount', '10')]
[('currencyname', 'Eastern Caribbean dollar'), ('countryCount', '10')]
[('currencyname', 'West African CFA franc'), ('countryCount', '9')]
[('currencyname', 'Yugoslav dinar'), ('countryCount', '8')]
[('currencyname', 'Australian dollar'), ('countryCount', '8')]
[('currencyname', 'Central African CFA franc'), ('countryCount', '8')]
[('c

Final query for this task

In [76]:
# write your final query
#from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
queryString = """
SELECT DISTINCT ?currencyname (COUNT(distinct ?countryname) AS ?countryCount) #counting in how many countries the currency used
WHERE {

   #looking for properties of country predicate
   ?s wdt:P17 ?country .
   ?country wdt:P38 ?currency .
   ?currency sc:name ?currencyname.
    ?country sc:name ?countryname .   

}

GROUP BY ?currencyname
HAVING(COUNT(distinct ?countryname)>=2)
ORDER BY DESC (?countryCount)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('currencyname', 'euro'), ('countryCount', '40')]
[('currencyname', 'pound sterling'), ('countryCount', '28')]
[('currencyname', 'United States dollar'), ('countryCount', '22')]
[('currencyname', 'Soviet ruble'), ('countryCount', '20')]
[('currencyname', 'French franc'), ('countryCount', '18')]
[('currencyname', 'dinar'), ('countryCount', '13')]
[('currencyname', 'peseta'), ('countryCount', '11')]
[('currencyname', 'Egyptian pound'), ('countryCount', '11')]
[('currencyname', 'Canadian dollar'), ('countryCount', '10')]
[('currencyname', 'ancient Chinese coinage'), ('countryCount', '10')]
[('currencyname', 'Russian ruble'), ('countryCount', '10')]
[('currencyname', 'Eastern Caribbean dollar'), ('countryCount', '10')]
[('currencyname', 'West African CFA franc'), ('countryCount', '9')]
[('currencyname', 'Yugoslav dinar'), ('countryCount', '8')]
[('currencyname', 'Australian dollar'), ('countryCount', '8')]
[('currencyname', 'Central African CFA franc'), ('countryCount', '8')]
[('c

## Task 7
Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

In [2]:
queryString = """
SELECT DISTINCT ?p ?pname ?o
WHERE {

   #looking for the population of Italy
     wd:Q38 ?p ?o .
   ?p sc:name ?pname .
    FILTER(REGEX(?pname, "[Pp]opulation*"^^xsd:string))

}

LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '60317000')]
1


In [3]:
#[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '60317000')]
queryString = """
SELECT DISTINCT ?sname ?oname
WHERE {
  ?s wdt:P1082 ?o ;
      sc:name ?sname .
      ?o sc:name ?oname 

}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [4]:
# write your queries
# counting the contries using the same currency
#from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
queryString = """
SELECT DISTINCT  ?currencyname (COUNT(distinct ?countryname) AS ?countryCount) #counting in how many countries the currency used
WHERE {

   #looking for properties of country predicate
   ?s wdt:P17 ?country .
   ?country wdt:P38 ?currency ;
      sc:name ?countryname .
      ?currency sc:name ?currencyname .

}

GROUP BY ?currencyname

ORDER BY DESC (?countryCount)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('currencyname', 'euro'), ('countryCount', '40')]
[('currencyname', 'pound sterling'), ('countryCount', '28')]
[('currencyname', 'United States dollar'), ('countryCount', '22')]
[('currencyname', 'Soviet ruble'), ('countryCount', '20')]
[('currencyname', 'French franc'), ('countryCount', '18')]
[('currencyname', 'dinar'), ('countryCount', '13')]
[('currencyname', 'peseta'), ('countryCount', '11')]
[('currencyname', 'Egyptian pound'), ('countryCount', '11')]
[('currencyname', 'Canadian dollar'), ('countryCount', '10')]
[('currencyname', 'ancient Chinese coinage'), ('countryCount', '10')]
10


Final query for this task

In [42]:
# write your final query


# write your queries
# counting the contries using the same currency
# wd:Q8142 -> Currency
#from task 2 [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
#[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population'), ('o', '60317000')]

queryString = """
SELECT DISTINCT ?IRI ?Currency  (SUM(?pop) AS ?Population) #counting in how many people use the currency 
WHERE {

   #looking for properties of country predicate
   ?IRI wdt:P31 wd:Q8142;
        wdt:P17 ?country.
        
   ?country wdt:P1082 ?pop . 
   
   ?IRI sc:name ?Currency.

}

Group by ?Currency ?IRI
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('IRI', 'http://www.wikidata.org/entity/Q184172'), ('Currency', 'French franc'), ('Population', '67180549')]
[('IRI', 'http://www.wikidata.org/entity/Q3778763'), ('Currency', 'assignation ruble'), ('Population', '485556621')]
[('IRI', 'http://www.wikidata.org/entity/Q25344'), ('Currency', 'Swiss franc'), ('Population', '8503939')]
[('IRI', 'http://www.wikidata.org/entity/Q718954'), ('Currency', 'Old Taiwan dollar'), ('Population', '23593794')]
[('IRI', 'http://www.wikidata.org/entity/Q2273865'), ('Currency', 'Italian Somaliland somalo'), ('Population', '11031386')]
[('IRI', 'http://www.wikidata.org/entity/Q1620502'), ('Currency', 'Catalan peseta'), ('Population', '46733038')]
[('IRI', 'http://www.wikidata.org/entity/Q2273855'), ('Currency', 'Italian Somaliland rupia'), ('Population', '100426000')]
[('IRI', 'http://www.wikidata.org/entity/Q56302633'), ('Currency', 'Ouguiya (1973-2017)'), ('Population', '5542397')]
[('IRI', 'http://www.wikidata.org/entity/Q3935770'), ('Currency'